In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from random import randint
import plotly.express as px
from scipy import integrate
import umap
from sklearn.preprocessing import StandardScaler

In [ ]:
data_path = "/home/lucymartin/Documents/kiln"
# create data with double header for firing no. and location of measurement
kiln_temp = pd.read_csv(data_path + "/Kiln_data.csv", header = [0,1])

In [ ]:
kiln_temp.head()

In [ ]:
# create and assign random colours to each firing number
colors = []
n = len(set(kiln_temp.columns))
for i in range(n):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

In [ ]:
# create dictionaries to plot by colour and location
cols = kiln_temp.columns.levels[0][:-1]
columns_and_colors = dict(zip(cols, colors))
marker_dict = {"Front Temp (°C)": "circle", "Main Temp (°C)": "cross", "Chimney Temp (°C)": "square", "Unnamed: 20_level_1":"circle"}

In [ ]:
# Plot data
fig = go.Figure()
all_cols = kiln_temp.columns[1:]
for measurement in all_cols:
   # print(measurement)
    fig.add_trace(go.Scatter(x = kiln_temp["Unnamed: 0_level_0", "Time (H)"], y = kiln_temp[measurement[0],\
                            measurement[1]], mode = 'markers', name = measurement[0] + "_" + measurement[1], \
                            marker = dict(color = columns_and_colors[measurement[0]], \
                                          symbol = marker_dict[measurement[1].strip()])))

fig.update_layout(title = "Kiln temperature time series", xaxis = dict(title = "Time (H)"),\
                  yaxis = dict(title = "Temperature (°C)"))
fig.show()

Start by look at:
- Time above 1200
- Time taken to reach 800
- Fluxuation (FT)

In [ ]:
time_to_thresh_low = 800
time_to_thresh_up = 1200
time_above_thresh = 1200

In [ ]:
# define empty lists to fill with time series properties
threshold_time_lower_tot = []
threshold_time_upper_tot = []
firing_id_array = []
time_above_tot = []
end_of_firing_tot = []
temp_work = []
for measurement in all_cols:
    ### TIME TO THRESHOLD LOWER
    # find the index at which threshold temp is reached
    temp_index = ((kiln_temp[measurement]>=time_to_thresh_low).idxmax())
    # find the corresponding time
    time_threshold_reached = kiln_temp["Unnamed: 0_level_0", "Time (H)"][temp_index]
    #print(time_threshold_reached)
    threshold_time_lower_tot.append(time_threshold_reached)
    
    ### TIME TO THRESHOLD UPPER
    # find the index at which threshold temp is reached
    temp_index = ((kiln_temp[measurement]>=time_to_thresh_up).idxmax())
    # find the corresponding time
    time_threshold_reached = kiln_temp["Unnamed: 0_level_0", "Time (H)"][temp_index]
    #print(time_threshold_reached)
    threshold_time_upper_tot.append(time_threshold_reached)
    
    ### ADD FIRING ID
    firing_id = measurement[0] + "_" + measurement[1]
    firing_id_array.append(firing_id)
    
    ### TEMPERATURE WORK
    #print(kiln_temp.interpolate(method = "linear", limit = 1)["Unnamed: 0_level_0", "Time (H)"])
    #print(kiln_temp.interpolate(method = "linear", limit = 1)[measurement].dropna().index)
    work = integrate.trapz(y = kiln_temp.interpolate(method = "linear", limit = 1)[measurement].dropna().values, x=kiln_temp["Unnamed: 0_level_0", "Time (H)"][kiln_temp.interpolate(method = "linear", limit = 1)[measurement].dropna().index])
    temp_work.append(work)
    
    ### TIME ABOVE THRESHOLD
    # need this to reflect different data spacing in different firings...
    approx_time_above = sum(np.array(kiln_temp[measurement]) > time_above_thresh)
    
    ### LENGTH OF FIRING + ACTUAL TIME ABOVE
    # create array of locations not contining nan values
    missing_data_logic = np.argwhere(~np.isnan(kiln_temp[measurement].values))
    # if statement needed as some of the firings currently have no data
    if len(missing_data_logic) > 0:
        end_of_firing = kiln_temp["Unnamed: 0_level_0", "Time (H)"][missing_data_logic[-1][0]]
        end_of_firing_tot.append(end_of_firing)
        #print((kiln_temp["Unnamed: 0_level_0", "Time (H)"][missing_data_logic[-2][0]] -  kiln_temp["Unnamed: 0_level_0", "Time (H)"][missing_data_logic[-3][0]]))
        # modifying approximate time above to include time point spacing
        approx_time_above = approx_time_above * (kiln_temp["Unnamed: 0_level_0", "Time (H)"][missing_data_logic[-2][0]] -  kiln_temp["Unnamed: 0_level_0", "Time (H)"][missing_data_logic[-3][0]])
        time_above_tot.append(approx_time_above)
    else:
        end_of_firing_tot.append(np.nan)
        time_above_tot.append(np.nan)

In [ ]:
d = {"Measurement ID": firing_id_array, "Time to 800°C (H)":threshold_time_lower_tot, "Time to 1200°C (H)": threshold_time_upper_tot,\
     "Time above 1200°C (approx, (H))": time_above_tot, "End of firing (H)": end_of_firing_tot, "Work done": temp_work}
summary_data = pd.DataFrame(d)

In [ ]:
summary_data["Firing"] = summary_data["Measurement ID"].str.split("_").str[0]
summary_data["Measurement location"] = summary_data["Measurement ID"].str.split("_").str[1]

In [ ]:
summary_data

In [ ]:
px.scatter(summary_data, x = "Time to 800°C (H)", y = "Time above 1200°C (approx, (H))", \
           color = "Firing", hover_data = ["Measurement ID"], \
           title = "Comparison of temperature profile in each firing",
          symbol = "Measurement location")

In [ ]:
for_umap = summary_data.drop(columns = ["Measurement ID", "Firing", "Measurement location"]).to_numpy()
reducer = umap.UMAP()
#scale data
scaled_data = StandardScaler().fit_transform(for_umap)
embedding_scaled = reducer.fit_transform(scaled_data)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = embedding_scaled[:, 0],
    y = embedding_scaled[:, 1],
    hovertext = summary_data["Measurement ID"].values,
    mode = "markers"))
fig.show()

Points to keep in mind after chat with Robin:
- Interested in the relationeship between the two temperatures in a single firing
- Interested in time to 1000 degress
- Up to approx 1100 degrees the temperature will fluxuate due to a firing cycle of approx 10 mins where fuel is added and burns, this firing cycle may vary in length. Because we only have a measurement every hour we might be able to see the ailasising of this signal? 
- at 1100 in some of the firings there is 45 mins to an hour reduction, where loads of wood is added, and the air holes close so that the temperature drops. This blackened wood is then cleared and the temperature pushed to above 1200
- as the kiln approaches a temperature of 1200 the temperature should start to fluxuate less as the pots start to aquire and radiate their own heat.
- Robin suspects that a steep, smooth climb in temperature to 1100 degrees is good, but is interested in working out which parameters are most important, and if it is possible to tell from data collected while firing that the firing will be rubbish and can't be redeemed. 
- It is difficult to measure the goodness of s firing. Robin can give a general indication of the quality of an individual firing, but different traits can be seen as good, e.g., ashing. This will occur at different places in the kiln and different features on the pots may depend on placement in the kiln as well as features of the temperature profile. 
- Because kiln is huge and tradiation the temperature recorded if just a proxy for temperature in the kiln. 
- Temperature work (Temp * time) is aslo of interest. 

To Do:
- cluster based on time series
- extract metrics and cluster based on metrics
- look into comparison methods for 2 time series

- Robin to make some sort of pot scoring metric

In [ ]:
from fastdtw import fastdtw

In [ ]:
#Going to have to throw some data away as some time series have data points every 30 mins, but most don't so
# will just look at the every hour data.
comparison_df = kiln_temp.iloc[::2, :]
# some still have missing values in these ranges, interpolate linearly
comparison_df = comparison_df.interpolate(method = "linear", limit = 1)
#for the linear comparison fill the empty data points after the firing has ended with 0s
euclid_comparison_df = comparison_df.fillna(0)

In [ ]:
# convert to df of floats for linear comparison, with 0 padding at the end
euclid_comparison_df = euclid_comparison_df.astype(float)

Clustering based on the time series data:
Actually difficult to do because of the differing lengths of the data and the differing frequencies. We could use something like fastdtw to map the data sets onto each other despite the difference in length, however, we would still have to filter the data so that it was all at the same frequency, for example, once per hour. This would then tell us about similarities between the firings regarless of how long the firing took. 
direct comparison is otherwise difficult due to the differing lenghts. Would it be worth trying to pad the data with a very different end value - for example 0s once the firing has ended? This way we're not really losing information on the length of the firing. 

For Euclidean comparison keep the zero padding at the end, for the fastdtw get rid of this to compare the different lenths

In [ ]:
import scipy.cluster.hierarchy as shc
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import pdist, squareform

# Importing the library
import seaborn as sns

In [ ]:
# list of the firing names
firings = kiln_temp.columns.levels[0][:-1]

In [ ]:
# compare all firings to each other
mat = []
for i in range(len(firings)):
    line = []
    for j in range(len(firings)):
        #print(j)
        vec1 = euclid_comparison_df[firings[i], "Main Temp (°C)"].to_numpy().reshape(1, -1)
        vec2 = euclid_comparison_df[firings[j], "Main Temp (°C)"].to_numpy().reshape(1, -1)
        #print(vec1)
        dist = cosine_distances(vec1, vec2)
        line.append(1-dist[0][0])
    #print(line)
    mat.append(line)

In [ ]:
# convert comparisons of firings into data frame form
mat = np.array(mat)
mat = pd.DataFrame(mat, columns = firings)

In [ ]:
# cluster the firings based on linear comparison
sns.clustermap(mat)

In [ ]:
# compare all firings to each other with warped time
warped_mat = []
for i in range(len(firings)):
    line = []
    for j in range(len(firings)):
        #print(j)
        # for warped time comparison drop the padding NaNs at the end
        vec1 = comparison_df[firings[i], "Main Temp (°C)"].dropna().to_numpy()#.reshape(1, -1)
        vec2 = comparison_df[firings[j], "Main Temp (°C)"].dropna().to_numpy()#.reshape(1, -1)
        #print(vec1)
        dist = fastdtw(vec1, vec2)[0]
        #print(fastdtw(vec1, vec2))
        line.append(dist)
    #print(line)
    warped_mat.append(line)

In [ ]:
# convert comparisons of firings into data frame form
warped_mat = np.array(warped_mat)
warped_mat = pd.DataFrame(warped_mat, columns = firings)

In [ ]:
# cluster the firings based on linear comparison
sns.clustermap(warped_mat)